In [51]:
import pandas as pd
import json
pd.set_option('display.max_columns', None)


with open('../data/gsw_box_scores_full.json', encoding="utf8") as file:
    game_data = json.load(file)

**Process each player's stats for a game as an individual row**

In [52]:
def process_player_data(team_data, team_type, date_et):
    players = team_data['players']
    team_tricode = team_data['teamTricode']
    processed_data = []

    for player in players:
        player_data = {
            "NAME": player.get('firstName') + ' ' + player.get('familyName'),
            "TEAM": team_tricode,
            "STATUS": team_type,
            "DATE": date_et,
        }

        player_data.update(player.get('statistics', {}))
        processed_data.append(player_data)

    return processed_data


def player_data_processor(game_data):
    all_players_data = []
    for game in game_data['games']:
        date_et = game['gameEt'][:10]
        away_team_data = process_player_data(game['awayTeam'], 'away', date_et)
        home_team_data = process_player_data(game['homeTeam'], 'home', date_et)
        all_players_data += away_team_data + home_team_data

    return all_players_data

**Create players stats spreadsheet**

In [53]:
all_players_data = player_data_processor(game_data=game_data)
df = pd.DataFrame(all_players_data)

df.to_excel('../data/gsw_box_score_player_stats.xlsx')

**Process each team's total stats for a game as an individual row**

In [54]:
def process_team_data(team_data, team_type, date_et):
    team_tricode = team_data['teamTricode']
    team_stats = {
        "TEAM": team_tricode,
        "STATUS": team_type,
        "DATE": date_et,
    }
    team_stats.update(team_data['statistics'])

    return team_stats


def team_data_processor(game_data):
    all_team_data = []
    for game in game_data['games']:
        date_et = game['gameEt'][:10]
        away_team_data = process_team_data(game['awayTeam'], 'away', date_et)
        home_team_data = process_team_data(game['homeTeam'], 'home', date_et)
        all_team_data.append(away_team_data)
        all_team_data.append(home_team_data)

    return all_team_data

**Create team stats spreadsheet**

In [55]:
all_teams_data = team_data_processor(game_data=game_data)
df2 = pd.DataFrame(all_teams_data)

df2.to_excel('../data/gsw_box_score_team_stats.xlsx')